# Part 1 : Understanding Ion-switching

In this time, I leave out basic introduction of ion-switching.  
If you want to learn it, please check following kernel.  
Ref:https://www.kaggle.com/mobassir/understanding-ion-switching-with-modeling/notebook  
  
rev6: use original signal data because of following discussion:https://www.kaggle.com/c/liverpool-ion-switching/discussion/152189

# imports

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

# Part 2 : Modeling


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import random
#from tqdm import tqdm
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import GroupKFold, StratifiedKFold #add
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

import os


# Any results you write to the current directory are saved as output.

In [3]:
# configurations and main hyperparammeters
EPOCHS = 150
NNBATCHSIZE = 32 #32
GROUP_BATCH_SIZE = 2000 #4000
SEED = 71
LR = 0.001 
SPLITS = 5

outdir = 'wavenet_models'
flip = False
noise = False


if not os.path.exists(outdir):
    os.makedirs(outdir)



def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [4]:
#Set google drive
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
# read data
def read_data():
    #train = pd.read_csv('drive/My Drive/train_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    #test  = pd.read_csv('drive/My Drive/test_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32})
    train = pd.read_csv('train.csv')
    test  = pd.read_csv('test.csv')
    
    #from https://www.kaggle.com/sggpls/wavenet-with-shifted-rfc-proba and
    # https://www.kaggle.com/c/liverpool-ion-switching/discussion/144645
    Y_train_proba = np.load("Y_train_proba.npy")
    Y_test_proba = np.load("Y_test_proba.npy")
    
    for i in range(11):
        train[f"proba_{i}"] = Y_train_proba[:, i]
        test[f"proba_{i}"] = Y_test_proba[:, i]
    
    #Y_train_proba=pd.DataFrame(Y_train_proba)
    #Y_test_proba=pd.DataFrame(Y_test_proba)
    #train = pd.concat([train,Y_train_proba],axis=1)
    #test = pd.concat([test,Y_test_proba],axis=1)
    
    sub  = pd.read_csv('sample_submission.csv')
    return train, test, sub

# create batches of 4000 observations
def batching(df, batch_size):
    #print(df)
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

# get lead and lags features
def lag_with_pct_change(df, windows):
    for window in windows:    
        df['signal_shift_pos_' + str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' + str(window)] = df.groupby('group')['signal'].shift(-1 * window).fillna(0)
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3])
    # create signal ** 2 (this is the new feature)
    df['signal_2'] = df['signal'] ** 2
    return df

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features


def split(GROUP_BATCH_SIZE=2000, SPLITS=5):
    print('Reading Data Started...')
    train, test, sample_submission = read_data()
    train, test = normalize(train, test)
    print('Reading and Normalizing Data Completed')
    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size=GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size=GROUP_BATCH_SIZE)
    train, test, features = feature_selection(train, test)
    print(train.head())
    print('Feature Engineering Completed...')

    target = ['open_channels']
    group = train['group']
    kf =StratifiedKFold(n_splits=SPLITS) # GroupKFold(n_splits=SPLITS)
    splits = [x for x in kf.split(train, train[target], group)]
    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])
        new_splits.append(new_split)
    target_cols = ['open_channels']
    print(train.head(), train.shape)
    train_tr = np.array(list(train.groupby('group').apply(lambda x: x[target_cols].values))).astype(np.float32)
    train = np.array(list(train.groupby('group').apply(lambda x: x[features].values)))
    test = np.array(list(test.groupby('group').apply(lambda x: x[features].values)))
    print(train.shape, test.shape, train_tr.shape)
    return train, test, train_tr, new_splits

# wavenet 

In [6]:
#from https://www.kaggle.com/hanjoonchoe/wavenet-lstm-pytorch-ignite-ver

class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

In [7]:
# from https://www.kaggle.com/hanjoonchoe/wavenet-lstm-pytorch-ignite-ver        
class Wave_Block(nn.Module):
    
    def __init__(self,in_channels,out_channels,dilation_rates):
        super(Wave_Block,self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        
        self.convs.append(nn.Conv1d(in_channels,out_channels,kernel_size=1))
        dilation_rates = [2**i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate,dilation=dilation_rate))
            self.gate_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate,dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=1))
            
    def forward(self,x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = F.tanh(self.filter_convs[i](x))*F.sigmoid(self.gate_convs[i](x))
            x = self.convs[i+1](x)
            x += res
        return x
    
    

    
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        input_size = 256#128 #256

        self.LSTM = nn.GRU(input_size=input_size,hidden_size=128,num_layers=2,batch_first=True,bidirectional=True)
        self.attention = Attention(input_size,2000) #add
        #self.rnn = nn.RNN(input_size, 64, 2, batch_first=True, nonlinearity='relu')
        self.wave_block1 = Wave_Block(19,16,12) #19 is a number of features
        self.wave_block2 = Wave_Block(16,32,8)
        self.wave_block3 = Wave_Block(32,64,4)
        self.wave_block4 = Wave_Block(64, 128, 2)
        self.wave_block5 = Wave_Block(128, 256, 1) #add
        self.dropout = nn.Dropout() #add
        self.fc = nn.Linear(256, 11)
            
    def forward(self,x):
        x = x.permute(0, 2, 1)
        
        x = self.wave_block1(x)
        x = self.wave_block2(x)
        x = self.wave_block3(x)
        
        #x,_ = self.LSTM(x)
        x = self.wave_block4(x)
        x = self.wave_block5(x) #add
        x = x.permute(0, 2, 1)
        x,_ = self.LSTM(x)
        #x = self.conv1(x)
        #print(x.shape)
        #x = self.rnn(x)
        #x = self.attention(x)
        x = self.dropout(x)#add
        x = self.fc(x)
        return x

   
    
class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt', is_maximize=True):
        self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
        self.counter, self.best_score = 0, None
        self.is_maximize = is_maximize


    def load_best_weights(self, model):
        model.load_state_dict(torch.load(self.checkpoint_path))

    def __call__(self, score, model):
        if self.best_score is None or \
                (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
            torch.save(model.state_dict(), self.checkpoint_path)
            self.best_score, self.counter = score, 0
            return 1
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return 2
        return 0

In [8]:
from torch.utils.data import Dataset, DataLoader
class IronDataset(Dataset):
    def __init__(self, data, labels, training=True, transform=None, seq_len=5000, flip=0.5, noise_level=0, class_split=0.0):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.training = training
        self.flip = flip
        self.noise_level = noise_level
        self.class_split = class_split
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.data[idx]
        labels = self.labels[idx]

        return [data.astype(np.float32), labels.astype(int)]

In [9]:
train, test, train_tr, new_splits = split()

Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
     time    signal  open_channels   proba_0   proba_1   proba_2   proba_3  \
0  0.0001 -1.242798              0  0.966731  0.028343  0.004812  0.000114   
1  0.0002 -1.271483              0  0.996045  0.003466  0.000426  0.000063   
2  0.0003 -1.137109              0  0.976313  0.018989  0.004677  0.000021   
3  0.0004 -1.356819              0  0.996002  0.003625  0.000326  0.000046   
4  0.0005 -1.360446              0  0.997465  0.002335  0.000158  0.000042   

   proba_4  proba_5  proba_6  proba_7  proba_8  proba_9  proba_10  group  \
0      0.0      0.0      0.0      0.0      0.0      0.0       0.0      0   
1      0.0      0.0      0.0      0.0      0.0      0.0       0.0      0   
2      0.0      0.0      0.0      0.0      0.0      0.0       0.0      0   
3      0.0      0.0      0.0      0.0      0.0      0.0       0.0      0   
4      0.0      0.0      0.0      0.0  

In [10]:
pip install torchcontrib

Note: you may need to restart the kernel to use updated packages.


In [11]:
from torchcontrib.optim import SWA
import torchcontrib

In [12]:
model = Classifier()
model

Classifier(
  (LSTM): GRU(256, 128, num_layers=2, batch_first=True, bidirectional=True)
  (attention): Attention()
  (wave_block1): Wave_Block(
    (convs): ModuleList(
      (0): Conv1d(19, 16, kernel_size=(1,), stride=(1,))
      (1): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (2): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (3): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (4): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (5): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (6): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (7): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (8): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (9): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (10): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (11): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (12): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
    )
    (filter_convs): ModuleList(
      (0): Conv1d(16, 16, kernel_size=(3,), strid

In [13]:
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.is_available())

10.1
1
True


In [14]:
%%time
test_y = np.zeros([int(2000000/GROUP_BATCH_SIZE), GROUP_BATCH_SIZE, 1])
test_dataset = IronDataset(test, test_y, flip=False)
test_dataloader = DataLoader(test_dataset, NNBATCHSIZE, shuffle=False, num_workers=8, pin_memory=True)
test_preds_all = np.zeros((2000000, 11))
train_preds_all = np.zeros((5000000, 11))

oof_score = []
for index, (train_index, val_index, _) in enumerate(new_splits[0:], start=0):
    print("Fold : {}".format(index))
    train_dataset = IronDataset(train[train_index], train_tr[train_index], seq_len=GROUP_BATCH_SIZE, flip=flip, noise_level=noise)
    train_dataloader = DataLoader(train_dataset, NNBATCHSIZE, shuffle=True, num_workers=8, pin_memory=True)

    valid_dataset = IronDataset(train[val_index], train_tr[val_index], seq_len=GROUP_BATCH_SIZE, flip=False)
    valid_dataloader = DataLoader(valid_dataset, NNBATCHSIZE, shuffle=False, num_workers=4, pin_memory=True)

    it = 0
    model = Classifier()
    model = model.cuda()

    early_stopping = EarlyStopping(patience=40, is_maximize=True,
                                   checkpoint_path=os.path.join(outdir, "gru_clean_checkpoint_fold_{}_iter_{}.pt".format(index,
                                                                                                             it)))

    weight = None#cal_weights()
    criterion = nn.CrossEntropyLoss(weight=weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    
    optimizer = torchcontrib.optim.SWA(optimizer, swa_start=10, swa_freq=5, swa_lr=0.002)


    schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.2)
    avg_train_losses, avg_valid_losses = [], []

    

    for epoch in range(EPOCHS):
        print('**********************************')
        print("Folder : {} Epoch : {}".format(index, epoch))
        print("Curr learning_rate: {:0.9f}".format(optimizer.param_groups[0]['lr']))
        train_losses, valid_losses = [], []
        tr_loss_cls_item, val_loss_cls_item = [], []

        model.train()  # prep model for training
        train_preds, train_true = torch.Tensor([]).cuda(), torch.LongTensor([]).cuda()#.to(device)
        
        for x, y in tqdm(train_dataloader):
            x = x.cuda()
            y = y.cuda()
            #print(x.shape)

            optimizer.zero_grad()
            #loss_fn(model(input), target).backward()

            #optimizer.zero_grad()
            predictions = model(x)

            predictions_ = predictions.view(-1, predictions.shape[-1])
            y_ = y.view(-1)
            

            loss = criterion(predictions_, y_)

            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            
            #schedular.step()
            # record training lossa
            train_losses.append(loss.item())
            train_true = torch.cat([train_true, y_], 0)
            train_preds = torch.cat([train_preds, predictions_], 0)

        #model.eval()  # prep model for evaluation
        optimizer.swap_swa_sgd()
        val_preds, val_true = torch.Tensor([]).cuda(), torch.LongTensor([]).cuda()
        print('EVALUATION')
        #pred_list = []#add
        with torch.no_grad():
            for x, y in tqdm(valid_dataloader):
                x = x.cuda()#.to(device)
                y = y.cuda()#..to(device)

                predictions = model(x)
                predictions_ = predictions.view(-1, predictions.shape[-1])
                y_ = y.view(-1)
                #pred_list.append(F.softmax(predictions_, dim=1).cpu().numpy()) #add
                #train_preds_ = np.vstack(pred_list) #get train predict data
                #train_preds_all += train_preds_ #get train predict data

                loss = criterion(predictions_, y_)
                valid_losses.append(loss.item())

                val_true = torch.cat([val_true, y_], 0)
                val_preds = torch.cat([val_preds, predictions_], 0)

        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        print("train_loss: {:0.6f}, valid_loss: {:0.6f}".format(train_loss, valid_loss))

        train_score = f1_score(train_true.cpu().detach().numpy(), train_preds.cpu().detach().numpy().argmax(1),
                               labels=list(range(11)), average='macro')

        val_score = f1_score(val_true.cpu().detach().numpy(), val_preds.cpu().detach().numpy().argmax(1),
                             labels=list(range(11)), average='macro')

        schedular.step(val_score)
        print("train_f1: {:0.6f}, valid_f1: {:0.6f}".format(train_score, val_score))
        res = early_stopping(val_score, model)
        #print('fres:', res)
        if  res == 2:
            print("Early Stopping")
            print('folder %d global best val max f1 model score %f' % (index, early_stopping.best_score))
            break
        elif res == 1:
            print('save folder %d global val max f1 model score %f' % (index, val_score))
    print('Folder {} finally best global max f1 score is {}'.format(index, early_stopping.best_score))
    oof_score.append(round(early_stopping.best_score, 6))
    
    model.eval()
    pred_list = []
    with torch.no_grad():
        for x, y in tqdm(test_dataloader):
            
            x = x.cuda()
            y = y.cuda()

            predictions = model(x)
            predictions_ = predictions.view(-1, predictions.shape[-1]) # shape [128, 4000, 11]
            print(predictions.shape, F.softmax(predictions_, dim=1).cpu().numpy().shape)
            pred_list.append(F.softmax(predictions_, dim=1).cpu().numpy()) # shape (512000, 11)
            #a = input()
        test_preds = np.vstack(pred_list) # shape [2000000, 11]
        test_preds_all += test_preds
    
    model.eval()
    pred_list = []
    with torch.no_grad():
        train_dataset = IronDataset(train, train_tr, seq_len=GROUP_BATCH_SIZE, flip=flip, noise_level=noise)
        train_dataloader = DataLoader(train_dataset, NNBATCHSIZE, shuffle=False, num_workers=8, pin_memory=True)
        for x, y in tqdm(train_dataloader):
            
            x = x.cuda()
            y = y.cuda()

            predictions = model(x)
            predictions_ = predictions.view(-1, predictions.shape[-1])
            print(predictions.shape, F.softmax(predictions_, dim=1).cpu().numpy().shape)
            pred_list.append(F.softmax(predictions_, dim=1).cpu().numpy())
            #a = input()
        train_preds_ = np.vstack(pred_list) 
        train_preds_all += train_preds_


Fold : 0
**********************************
Folder : 0 Epoch : 0
Curr learning_rate: 0.001000000



EVALUATION



train_loss: 0.843548, valid_loss: 0.459308
train_f1: 0.526469, valid_f1: 0.620080
save folder 0 global val max f1 model score 0.620080
**********************************
Folder : 0 Epoch : 1
Curr learning_rate: 0.002000000



EVALUATION



train_loss: 0.176108, valid_loss: 0.169176
train_f1: 0.871095, valid_f1: 0.858222
save folder 0 global val max f1 model score 0.858222
**********************************
Folder : 0 Epoch : 2
Curr learning_rate: 0.002000000


...


train_loss: 0.070060, valid_loss: 0.075949
train_f1: 0.942513, valid_f1: 0.942094
save folder 4 global val max f1 model score 0.942094
**********************************
Folder : 4 Epoch : 149
Curr learning_rate: 0.002000000



EVALUATION



train_loss: 0.070544, valid_loss: 0.073848
train_f1: 0.942143, valid_f1: 0.941845
Folder 4 finally best global max f1 score is 0.9420938476756344


torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]

torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]) (64000, 11)
torch.Size([32, 2000, 11]

In [15]:
# -*- coding: utf-8 -*-
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #over 10000 byte data pick up
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                     test| 304000128|
|               test_preds|  88000112|
|           test_preds_all| 176000112|
|                   test_y|  16000128|
|                    train| 760000128|
|             train_preds_| 220000112|
|          train_preds_all| 440000112|
|                 train_tr|  20000128|


In [16]:
print('all folder score is:%s'%str(oof_score))
print('OOF mean score is: %f'% (sum(oof_score)/len(oof_score)))
print('Generate submission.............')
submission_csv_path = 'sample_submission.csv'
ss = pd.read_csv(submission_csv_path, dtype={'time': str})
test_preds_all = test_preds_all / np.sum(test_preds_all, axis=1)[:, None]
test_pred_frame = pd.DataFrame({'time': ss['time'].astype(str),
                                'open_channels': np.argmax(test_preds_all, axis=1)})
test_pred_frame.to_csv("submission_wavenet_rev6.csv", index=False)

print('over')

all folder score is:[0.94049, 0.941019, 0.951834, 0.944893, 0.942094]
OOF mean score is: 0.944066
Generate submission.............
over


In [ ]:
train = pd.read_csv('train.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
train_preds_all = train_preds_all / np.sum(train_preds_all, axis=1)[:, None]
train["y_pred"]=np.argmax(train_preds_all, axis=1)

train.to_csv("wavenet_rev6_y_pred.csv", index=False)

print('over')

over
